<a href="https://colab.research.google.com/github/OsvaldoMS1982/UNIR/blob/main/ComparativaModelos1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

# Montar Google Drive
drive.mount('/content/drive')

# Definir la ruta de Spider en Google Drive
SPIDER_PATH = "/content/drive/MyDrive/spider"

# Verificar si los archivos existen
assert os.path.exists(SPIDER_PATH), f"Error: La carpeta Spider no se encontró en {SPIDER_PATH}. Verifica la ruta en tu Google Drive."

print("Google Drive montado correctamente. Spider dataset encontrado.")

Mounted at /content/drive
Google Drive montado correctamente. Spider dataset encontrado.


In [34]:
import json

# Ruta al archivo dev.json
dev_json_path = os.path.join(SPIDER_PATH, "dev.json")

# Verificar si el archivo existe
assert os.path.exists(dev_json_path), f"Error: {dev_json_path} no existe. Verifica que el dataset Spider esté completo."

# Cargar el dataset
with open(dev_json_path, "r", encoding="utf-8") as f:
    spider_data = json.load(f)

print(f"Dataset cargado correctamente con {len(spider_data)} ejemplos.")

Dataset cargado correctamente con 1034 ejemplos.


In [3]:
pip install transformers datasets nltk torch torchvision torchaudio pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [35]:
import torch
import torchvision
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration
from nltk.translate.bleu_score import sentence_bleu
import sqlite3
import time
import random
import numpy as np
import pandas as pd

In [6]:
# Espacios para otros modelos (deben implementarse)
def generate_sql_sqlnet(question):
    return "SELECT * FROM dummy_table", 0.5  # Simulación

def generate_sql_ratsql(question):
    return "SELECT * FROM dummy_table", 0.6  # Simulación

def generate_sql_tablebert(question):
    return "SELECT * FROM dummy_table", 0.7  # Simulación

In [15]:
# Cargar el tokenizador y modelo T5
tokenizer_t5 = T5Tokenizer.from_pretrained("t5-small")
model_t5 = T5ForConditionalGeneration.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
def generate_sql_t5(question):
    inputs = tokenizer_t5(f"translate English to SQL: {question}", return_tensors="pt")
    start_time = time.time()
    outputs = model_t5.generate(**inputs)
    end_time = time.time()

    generated_sql = tokenizer_t5.decode(outputs[0], skip_special_tokens=True)
    inference_time = end_time - start_time
    return generated_sql, inference_time

In [17]:
def evaluate_bleu(predicted_sql, reference_sql):
    reference_tokens = [reference_sql.split()]
    predicted_tokens = predicted_sql.split()
    return sentence_bleu(reference_tokens, predicted_tokens)

In [18]:
def exact_match(predicted, reference):
    return predicted.strip().lower() == reference.strip().lower()

In [19]:
def execute_sql(database_path, sql_query):
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()
    except Exception as e:
        result = str(e)
    conn.close()
    return result

In [32]:
# Base de datos de Spider (cambia la ruta si es necesario)
database_path = os.path.join("/content/drive/MyDrive/spider", "database", "example.db")

In [33]:
# Lista de modelos a evaluar
models = {
    "T5": generate_sql_t5,
    "SQLNet": generate_sql_sqlnet,
    "RAT-SQL": generate_sql_ratsql,
    "Table-BERT": generate_sql_tablebert
}

In [39]:
# Evaluación
results = []
num_samples = 10  # Para evitar que tarde demasiado, usar solo 10 ejemplos

for item in random.sample(spider_data, num_samples):
    question = item["question"]
    reference_sql = item["query"]

    for model_name, model_func in models.items():
        predicted_sql, inference_time = model_func(question)

        execution_accuracy = execute_sql(database_path, predicted_sql) == execute_sql(database_path, reference_sql)
        bleu_score = evaluate_bleu(predicted_sql, reference_sql)
        exact_match_score = exact_match(predicted_sql, reference_sql)

        results.append({
            "Modelo": model_name,
            "Pregunta": question,
            "SQL Referencia": reference_sql,
            "SQL Predicho": predicted_sql,
            "BLEU Score": bleu_score,
            "Exact Match": exact_match_score,
            "Execution Accuracy": execution_accuracy,
            "Tiempo de Inferencia (s)": inference_time
        })

# Convertir a DataFrame
df_results = pd.DataFrame(results)

# Mostrar tabla
#import ace_tools as tools
#tools.display_dataframe_to_user(name="Resultados Comparación Modelos", dataframe=df_results)

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [40]:
import pandas as pd

# Mostrar los resultados en consola
print(df_results)

# O guardarlos en un archivo CSV para análisis posterior
df_results.to_csv("resultados_comparacion.csv", index=False)

# O si usas Jupyter Notebook, usa:
from IPython.display import display
display(df_results)

        Modelo                                           Pregunta  \
0           T5  What are the record companies of orchestras in...   
1       SQLNet  What are the record companies of orchestras in...   
2      RAT-SQL  What are the record companies of orchestras in...   
3   Table-BERT  What are the record companies of orchestras in...   
4           T5             How many flights have destination ATO?   
5       SQLNet             How many flights have destination ATO?   
6      RAT-SQL             How many flights have destination ATO?   
7   Table-BERT             How many flights have destination ATO?   
8           T5  What are all the course names of the courses w...   
9       SQLNet  What are all the course names of the courses w...   
10     RAT-SQL  What are all the course names of the courses w...   
11  Table-BERT  What are all the course names of the courses w...   
12          T5  Find the model of the car whose weight is belo...   
13      SQLNet  Find the model of 

,Modelo,Pregunta,SQL Referencia,SQL Predicho,BLEU Score,Exact Match,Execution Accuracy,Tiempo de Inferencia (s)
0,T5,What are the record companies of orchestras in...,SELECT Record_Company FROM orchestra ORDER BY ...,Was sind die Plattenfirmen von Orchestern in d...,0.000000e+00,False,False,0.810179
1,SQLNet,What are the record companies of orchestras in...,SELECT Record_Company FROM orchestra ORDER BY ...,SELECT * FROM dummy_table,5.635810e-232,False,False,0.500000
2,RAT-SQL,What are the record companies of orchestras in...,SELECT Record_Company FROM orchestra ORDER BY ...,SELECT * FROM dummy_table,5.635810e-232,False,False,0.600000
3,Table-BERT,What are the record companies of orchestras in...,SELECT Record_Company FROM orchestra ORDER BY ...,SELECT * FROM dummy_table,5.635810e-232,False,False,0.700000
4,T5,How many flights have destination ATO?,SELECT count(*) FROM FLIGHTS WHERE DestAirport...,Wie viele Flüge haben Ziel ATO?,0.000000e+00,False,False,0.401983
5,SQLNet,How many flights have destination ATO?,SELECT count(*) FROM FLIGHTS WHERE DestAirport...,SELECT * FROM dummy_table,5.635810e-232,False,False,0.500000
6,RAT-SQL,How many flights have destination ATO?,SELECT count(*) FROM FLIGHTS WHERE DestAirport...,SELECT * FROM dummy_table,5.635810e-232,False,False,0.600000
7,Table-BERT,How many flights have destination ATO?,SELECT count(*) FROM FLIGHTS WHERE DestAirport...,SELECT * FROM dummy_table,5.635810e-232,False,False,0.700000
8,T5,What are all the course names of the courses w...,SELECT DISTINCT T1.course_name FROM Courses AS...,"Was sind alle Kursnamen der Kurse, in denen si...",0.000000e+00,False,False,0.728423
9,SQLNet,What are all the course names of the courses w...,SELECT DISTINCT T1.course_name FROM Courses AS...,SELECT * FROM dummy_table,9.793569e-233,False,False,0.500000


In [43]:
df_grouped = df_results.groupby("Modelo").mean()
print(df_grouped)

TypeError: agg function failed [how->mean,dtype->object]